In [1]:
import os
from pathlib import Path
import logging

from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn as nn 
import torch.optim as optim
from torchinfo import summary

import config
import dataset
import dataset_fasdd
import models
import loss
import val_epoch
import utils

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


# Logger

In [2]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('BED Classifier trained with FASDD evaluated with several loaders.\n')  

# Validation Dataset

In [3]:
# VALIDATION DATASET
val_transform = A.Compose([
    A.Resize(config.IMG_H, config.IMG_W, p=1),
    ToTensorV2(p=1),
    ]
)

### DFire

In [4]:
print("\nTEST DFire dataset")
val_dfire_dataset = dataset.DFireDataset(
    img_h = config.IMG_H,
    img_w = config.IMG_W,
    img_dir = config.VAL_IMG_DIR,
    label_dir = config.VAL_LABEL_DIR,
    num_classes = config.N_CLASSES,
    ds_len = config.DS_LEN,
    transform=val_transform)
print(f'\nTest dataset len: {len(val_dfire_dataset)}')
logger.info("\nTEST DFire dataset")
logger.info(f'Test dataset len: {len(val_dfire_dataset)}')


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306


In [5]:
val_dfire_loader = DataLoader(
    dataset=val_dfire_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    shuffle=False,
    drop_last=True)

### FASDD UAV

In [6]:
print("\nTEST FASDD UAV dataset")
val_fasdd_uav_ds = dataset_fasdd.FASDDDataset(
    img_h=config.IMG_H, 
    img_w=config.IMG_W, 
    imgs_dir=config.FASDD_UAV_IMGS_DIR, 
    labels_file=config.FASDD_UAV_TEST_LABELS_FILE, 
    num_classes=config.N_CLASSES,
    ds_len=config.DS_LEN,
    transform=val_transform)
print(f'\nTest FASDD UAV dataset len: {len(val_fasdd_uav_ds)}')
logger.info("\nTEST FASDD UAV dataset")
logger.info(f'Test FASDD UAV dataset len: {len(val_fasdd_uav_ds)}')


TEST FASDD UAV dataset
DFire Removed wrong images: 0
DFire empty images: 1997
DFire only smoke images: 846
DFire only fire images: 35
DFire smoke and fire images: 1303

Test FASDD UAV dataset len: 4181


In [7]:
val_fasdd_uav_loader = DataLoader(
    dataset=val_fasdd_uav_ds,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    shuffle=False,
    drop_last=True)

### FASDD CV

In [8]:
print("\nTEST FASDD CV dataset")
val_fasdd_cv_ds = dataset_fasdd.FASDDDataset(
    img_h=config.IMG_H, 
    img_w=config.IMG_W, 
    imgs_dir=config.FASDD_CV_IMGS_DIR, 
    labels_file=config.FASDD_CV_TEST_LABELS_FILE, 
    num_classes=config.N_CLASSES,
    ds_len=config.DS_LEN,
    transform=val_transform)
print(f'\nTest FASDD CV dataset len: {len(val_fasdd_cv_ds)}')
logger.info("\nTEST FASDD CV dataset")
logger.info(f'Test FASDD CV dataset len: {len(val_fasdd_cv_ds)}')


TEST FASDD CV dataset
DFire Removed wrong images: 0
DFire empty images: 6533
DFire only smoke images: 3902
DFire only fire images: 2091
DFire smoke and fire images: 3358

Test FASDD CV dataset len: 15884


In [9]:
val_fasdd_cv_loader = DataLoader(
    dataset=val_fasdd_cv_ds,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    shuffle=False,
    drop_last=True)

# Full DS Contatenation

In [10]:
print("Concatenate Test DFire and FASDD UAV datasets")
logger.info("Concatenate Test DFire and FASDD UAV datasets")
val_ds_concat = torch.utils.data.ConcatDataset((val_dfire_dataset, val_fasdd_uav_ds))
print(f'Test dataset len: {len(val_ds_concat)}')
logger.info(f'Test dataset len: {len(val_ds_concat)}')

print("Concatenate with FASDD CV dataset")
logger.info("Concatenate with FASDD CV dataset")
val_ds = torch.utils.data.ConcatDataset((val_ds_concat, val_fasdd_cv_ds))
print(f'Test dataset len: {len(val_ds)}')
logger.info(f'Test dataset len: {len(val_ds)}')

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 8487
Concatenate with FASDD CV dataset
Test dataset len: 24371


In [11]:
val_loader = DataLoader(dataset=val_ds,
                        batch_size=config.BATCH_SIZE,
                        num_workers=config.NUM_WORKERS,
                        pin_memory=config.PIN_MEMORY,
                        shuffle=False,
                        drop_last=True)

# Loss Function

In [12]:
# LOSS FUNCTION
if config.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    print(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config.DEVICE, smoke_precision_weight=config.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


# Setup Model and Load Weights

In [13]:
if config.MODEL == "BED":
    print("Using BED Classifier")
    model = models.BED_CLASSIFIER(num_classes=config.N_CLASSES).to(config.DEVICE)  
else:
    print("Wrong Model")
    raise SystemExit("Wrong Model")

Using BED Classifier


In [14]:
optimizer = optim.Adam(
    model.parameters(), 
    lr=config.LEARNING_RATE, 
    weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',
    factor=config.FACTOR, 
    patience=config.PATIENCE, 
    threshold=config.THRES, 
    threshold_mode='abs',
    min_lr=config.MIN_LR)

### Model trained with DFire and FASDD

In [15]:
model_name = 'BED_classifier__best_smoke__precision=0.935__epoch=87.pt'
model_path = 'experiments_256_add_fasdd/test_00/weights/' + model_name
epoch_saved = utils.load_checkpoint(model_path, model, optimizer, scheduler, config.DEVICE)

Loading Model. Trained during 87 epochs


# Evaluate Model Trained with FASDD with Different Loaders

In [16]:
model.eval()

with torch.no_grad():
    print("____________________________ Full DS ____________________________")
    logger.info("____________________________ Full DS ____________________________")
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)
    logger.info(val_metrics)
    print("\n____________________________ DFire ____________________________")
    logger.info("\n____________________________ DFire ____________________________")
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_dfire_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)
    logger.info(val_metrics)
    print("\n____________________________ FASDD UAV ____________________________")
    logger.info("\n____________________________ FASDD UAV ____________________________")
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_fasdd_uav_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)
    logger.info(val_metrics)
    print("\n____________________________ FASDD CV ____________________________")
    logger.info("\n____________________________ FASDD UAV ____________________________")
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_fasdd_cv_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)
    logger.info(val_metrics)

____________________________ Full DS ____________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:22<00:00, 16.55it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
19.195      |12.298      |6.897       
SMOKE -> Precision: 0.935 - Recall: 0.878 - Accuracy: 0.914 - F1: 0.906
FIRE -> Precision: 0.913 - Recall: 0.974 - Accuracy: 0.961 - F1: 0.942

____________________________ DFire ____________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:02<00:00, 27.22it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
17.534      |11.831      |5.703       
SMOKE -> Precision: 0.934 - Recall: 0.908 - Accuracy: 0.925 - F1: 0.921
FIRE -> Precision: 0.940 - Recall: 0.945 - Accuracy: 0.970 - F1: 0.942

____________________________ FASDD UAV ____________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:10<00:00,  6.42it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
9.396       |7.061       |2.335       
SMOKE -> Precision: 0.997 - Recall: 0.920 - Accuracy: 0.957 - F1: 0.957
FIRE -> Precision: 0.980 - Recall: 0.972 - Accuracy: 0.985 - F1: 0.976

____________________________ FASDD CV ____________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:10<00:00, 23.12it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
22.216      |13.792      |8.424       
SMOKE -> Precision: 0.917 - Recall: 0.857 - Accuracy: 0.899 - F1: 0.886
FIRE -> Precision: 0.892 - Recall: 0.981 - Accuracy: 0.953 - F1: 0.935
